# 使用交叉验证法进行分析(Cross Validate)

In [190]:
from sklearn import datasets            #鸢尾花数据集被sklearn的datasets所包含，需要引用
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC # Support Vector Cluster
import numpy as np

iris_dataset = datasets.load_iris()
iris_data = iris_dataset.data
iris_label = iris_dataset.target
print iris_data[:6]
print iris_label[:6]
'''
将鸢尾花数据iris_data分成训练集x_train和测试集x_test
将鸢尾花种类iris_label分成训练集y_train和测试集y_test
test_size=0.2,测试集数据占总数据的0.2倍
'''
x_train,x_test,y_train,y_test = train_test_split(iris_data, iris_label,test_size=0.2)

'''
创建支持向量机分类器，其中核函数为rbf(径向基函数)
除此之外，还有linear(线性核函数)、poly(多项式核函数模型)
'''
clf = SVC(kernel='rbf')

'''
将训练集数据输入模型进行训练
'''
clf.fit(x_train, y_train)

'''
使用训练好的模型预测结果
'''
predict = clf.predict(x_test)
print predict.shape
'''
#将预测结果与实际值比较，并统计数组中True的个数即为正确个数
'''
right = sum(predict==y_test)

print 'Precision {}%'.format(right*100.0 / predict.shape[0])

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]
 [ 5.4  3.9  1.7  0.4]]
[0 0 0 0 0 0]
(30L,)
Precision 93.3333333333%


# 使用留一验证法进行分析(One-Left-Algorithm)

由于交叉验证法每次选取的测试集是随机的，因此每次运算结果未必相同。此时可采用留一验证法进行分析。

选取第0组作为测试集，其它为训练集重新训练模型，对第0组数据进行预测并验证是否正确。

选取第1组作为测试集，其它为训练集重新训练模型，对第1组数据进行预测并验证是否正确。

...

选取第n-1组作为测试集，其它为训练集重新训练模型，对第n-1组数据进行预测并验证是否正确。

统计这n组结果的准确率

In [191]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
import numpy as np

def data_svc_test(data, label, index):
    #除第index号之外的数据为训练集
    x_train = np.vstack((data[0:index] , data[index + 1:-1]))
    #index为测试集
    x_test = data[index]
    
    y_train = np.hstack((label[0:index], label[index + 1:-1]))
    y_test = label[index]
    
    clf = SVC(kernel='rbf')
    clf.fit(x_train, y_train)
    predict = clf.predict(x_test)
    return predict == y_test


iris_dataset = datasets.load_iris()
iris_data = iris_dataset.data
iris_label = iris_dataset.target

length = iris_label.shape[0]
right = 0

for i in range(0,length):
    right += data_svc_test(iris_data, iris_label, i)[0]

print  'Precision {}%'.format((right*100.0 / length))   

Precision 98.0%


D:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
D:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
D:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

# 回归分析与波士顿房价数据集

## 什么是支持向量回归(Support Vector Regression)

回归分析是确定两种或两种以上变量间相互依赖的定量关系的一种统计分析方法。比如在中学阶段我们就学过通过最小二乘法回归分析对若干组变量进行线性拟合。现在我们将借助支持向量机进行回归分析。

支持向量其实是表示n维空间内某个n-1维超平面的一组变量。若干组由n个变量组成的数据项，都可以看作n维空间内的点。

现在拟合一个n－1维的超平面，使这些点尽可能靠近这个超平面，则该超平面为这些点的支持向量回归面。只要确定n个变量中的任意n－1个变量，就能借助该超平面确定剩余的一个变量。该方法被称为支持向量回归法。

在**支持向量机分类**时，支持向量要尽可能把不同标签的点分隔开。

而在**支持向量回归**时，则要让支持向量尽可能靠近所有点。

In [192]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVR
import numpy as np
from sklearn.preprocessing import StandardScaler

house_dataset = datasets.load_boston()
house_data = house_dataset.data
house_price = house_dataset.target
#print house_data[:5]
#print house_price[:5]
'''
预处理数据，
'''
x_train,x_test,y_train,y_test=train_test_split(house_data,house_price,test_size=0.2)
'''
StandardScaler是数据放缩的其中一种方案，即f(x)=(x - 平均值)/标准差
'''
scaler = StandardScaler()
'''
在数据模型建立时，只能使用训练组数据，因此使用训练组数据的标准差进行标准化
'''
scaler.fit(x_train)
'''
对训练组和测试组的输入数据进行标准化，
即f(训练组或测试组数据)=(训练组或测试组数据-训练组或测试组平均值)/训练组标准差
'''
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
'''
构建基于rbf（径向基函数）的SVR模型
'''
svr = SVR(kernel='rbf')
'''
训练组数据输入训练
'''
svr.fit(x_train, y_train)

y_predict = svr.predict(x_test)

'''
将实际结果与预测结果对比观察，
2列的数组，左边列是实际结果，右边列是预测结果
'''
result = np.hstack((y_test.reshape(-1,1), y_predict.reshape(-1,1)))
print result[:5]

[[ 19.4         20.36727332]
 [ 15.2         16.6761319 ]
 [  8.4         11.82764039]
 [ 15.6         15.25649279]
 [ 13.4         19.61272934]]


为了便于后续训练，需要对读取到的数据进行处理。影响房价的数据的范围都不一致，这些数据都不在一个数量级上，徜若不分青红皂白的进行训练，很容易导致数值大的数据对结果影响极大，从而不能平衡的体现出各个数据的重要性。

因此需要通过数学方法，依据方差、平均值等因素，把各类数据放缩到一个相同的范围内，使其影响力所占权重相近。

In [193]:
print a.ndim # 向量维度
print a.size # 向量所含元素总数量
print a.dtype # 每个元素的数据类型
print a.itemsize # 每个元素所占字节大小
print type(a)
print a.data

2
9
float64
8
<type 'numpy.ndarray'>
      �?      �?      �?      �?      �?      �?      �?      �?      �?


In [194]:
c = np.array([[3,7],[4,9]], dtype=complex)
print c

[[ 3.+0.j  7.+0.j]
 [ 4.+0.j  9.+0.j]]


In [195]:
print np.arange(3,9,2)
from numpy import pi
print np.linspace(7,20,9)

[3 5 7]
[  7.      8.625  10.25   11.875  13.5    15.125  16.75   18.375  20.   ]


In [196]:
a = np.ones((2,3),dtype=np.int)
print a
b = np.random.random((2,3))
print b

a*=3
print a
b+=a
print b


[[1 1 1]
 [1 1 1]]
[[ 0.69241504  0.48538016  0.58781734]
 [ 0.33028871  0.08934818  0.58947355]]
[[3 3 3]
 [3 3 3]]
[[ 3.69241504  3.48538016  3.58781734]
 [ 3.33028871  3.08934818  3.58947355]]


In [197]:
a=np.ones(3,dtype=np.int32)
b=np.linspace(0,pi,3)
print b.dtype
c=a+b
print c.dtype
d=np.exp(c*1j)
print d
print d.dtype

float64
float64
[ 0.54030231+0.84147098j -0.84147098+0.54030231j -0.54030231-0.84147098j]
complex128


In [198]:
#a = np.random.random((2,3))
a = np.ones((2,3))
print a
print a.sum()
print a.min()
print a.max()
print a.sum(axis=0)
print a.max(axis=1)
print '--------------------'
print a.cumsum(axis=0)

[[ 1.  1.  1.]
 [ 1.  1.  1.]]
6.0
1.0
1.0
[ 2.  2.  2.]
[ 1.  1.]
--------------------
[[ 1.  1.  1.]
 [ 2.  2.  2.]]


In [199]:
# 文本分词
# hash
# hash,weight
# hash-weight -> np.array
# sum(axis=1)

In [200]:
a = np.arange(10)**3
print a
print a[2:5]
print a[:6:2]
print a[::-1]

[  0   1   8  27  64 125 216 343 512 729]
[ 8 27 64]
[ 0  8 64]
[729 512 343 216 125  64  27   8   1   0]


In [201]:
def f(x,y): return 10*x+y
a = np.fromfunction(f,(5,4),dtype=int)
print a
print a[2,3]
print a[0:5,1]
print a[1:3,:]

[[ 0  1  2  3]
 [10 11 12 13]
 [20 21 22 23]
 [30 31 32 33]
 [40 41 42 43]]
23
[ 1 11 21 31 41]
[[10 11 12 13]
 [20 21 22 23]]


In [202]:
c=np.array([[[0,1,2],[10,12,13]],[[100,101,102],[110,112,113]]])
print(c.shape)

(2L, 2L, 3L)


In [203]:
a=np.ones((3,3))
b=np.ones((3,3))
c=np.vstack((a,b))
print c
c=np.hstack((a,b))
print c

print '-------------'

c=np.hsplit(a,3)
print c

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]]
-------------
[array([[ 1.],
       [ 1.],
       [ 1.]]), array([[ 1.],
       [ 1.],
       [ 1.]]), array([[ 1.],
       [ 1.],
       [ 1.]])]


In [204]:
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()
print iris.data[:5]

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]


In [205]:
from sklearn import svm
from sklearn import datasets
from sklearn.externals import joblib

digits = datasets.load_digits()
print dir(svm)
cls = svm.SVC(gamma=0.001,C=100.)
cls.fit(digits.data[:10], digits.target[:10])

print dir(joblib)
joblib.dump(cls,'digits.pkl')
cls1=joblib.load('digits.pkl')
cls1.predict(digits.data[-4:])

['LinearSVC', 'LinearSVR', 'NuSVC', 'NuSVR', 'OneClassSVM', 'SVC', 'SVR', '__all__', '__builtins__', '__doc__', '__file__', '__name__', '__package__', '__path__', 'base', 'bounds', 'classes', 'l1_min_c', 'liblinear', 'libsvm', 'libsvm_sparse']
['Logger', 'MemorizedResult', 'Memory', 'Parallel', 'PrintTime', '__builtins__', '__doc__', '__file__', '__name__', '__package__', '__path__', '__version__', '_compat', '_memory_helpers', '_multiprocessing_helpers', 'cpu_count', 'delayed', 'disk', 'dump', 'format_stack', 'func_inspect', 'hash', 'hashing', 'load', 'logger', 'memory', 'my_exceptions', 'numpy_pickle', 'parallel', 'pool']


array([0, 6, 9, 8])

In [206]:
from sklearn import random_projection
r = np.random.RandomState(0)
X = r.rand(10,2000)
X = np.array(X, dtype='float32')
trans = random_projection.GaussianRandomProjection()
X_new = trans.fit_transform(X)
print X_new

[[-0.16045204  0.75176081  0.2092798  ..., -0.49996248  0.49858932
   0.56963746]
 [ 0.91823786  0.5910029   0.53161775 ...,  0.18465406  0.39188501
   0.68389086]
 [ 0.08016833  0.68166798  0.16887302 ..., -0.10259364  0.26563237
   0.47546771]
 ..., 
 [ 0.79406512  0.57878564  0.08436274 ..., -0.44752587  0.70755114
   0.7535007 ]
 [ 0.32699259  0.41863856 -0.15044537 ..., -0.85340321  0.51504119
   0.2759975 ]
 [ 0.40669448  0.28638969  0.21575085 ..., -0.48905629 -0.02025767
  -0.04187596]]


In [207]:
import numpy as np
from sklearn.svm import SVC
rng = np.random.RandomState(0)
X = rng.rand(100, 10)
y = rng.binomial(1, 0.5, 100)
X_test = rng.rand(5, 10)
clf = SVC()
clf.set_params(kernel='linear').fit(X,y)
print clf.predict(X_test)

clf.set_params(kernel='rbf').fit(X,y)
print clf.predict(X_test)

[1 0 1 1 0]
[0 0 0 1 0]
